# Workshop 3: Exploring NT Scenario Results, Modifying Assumptions, and Benchmarking

:::{note} At the end of this notebook, you will be able to:

- Navigate and analyze NT scenario results using PyPSA-Explorer
- Interpret discrepancies between model outputs and TYNDP 2024 NT scenario
- Compare 2030 and 2040 scenarios interactively
- Modify model assumptions and generate new scenario results
- Apply the benchmarking framework to identify areas for model improvement

:::

:::{note}
If you have not set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work, download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the required packages by executing the following command in a Jupyter cell at the top of the notebook:

```sh
!pip install pypsa pypsa-explorer pandas matplotlib numpy
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa pypsa-explorer pandas matplotlib numpy

In [ ]:
import os
from datetime import datetime
import pandas as pd
import pypsa
import zipfile
from urllib.request import urlretrieve
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError
from IPython.display import Code, display
import matplotlib.pyplot as plt
from pypsa_explorer import create_app
from pathlib import Path

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False
plt.rcParams["figure.figsize"] = [14, 7]

In [ ]:
def unzip_with_timestamps(zip_path, extract_to):
    """Unzip a file while preserving original file timestamps."""
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for member in zip_ref.infolist():
            # Extract the file
            zip_ref.extract(member, extract_to)

            # Get the extracted file path
            extracted_path = os.path.join(extract_to, member.filename)

            # Get the modification time from the zip file
            date_time = datetime(*member.date_time)
            timestamp = date_time.timestamp()

            # Set both access and modification times
            os.utime(extracted_path, (timestamp, timestamp))

In [ ]:
urls = {
    "data/20251129_results.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-03/20251129_results.zip",
    "data/open-tyndp.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-03/open-tyndp-20251129.zip",
    "scripts/_helpers.py": "https://raw.githubusercontent.com/open-energy-transition/open-tyndp-workshops/refs/heads/feature/workshop-03-notebook/open-tyndp-workshops/scripts/_helpers.py",  # TODO: Update for version on main
}

os.makedirs("data", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/20251129_results"
if not os.path.exists(to_dir):
    print(f"Unzipping data/20251129_results.zip.")
    unzip_with_timestamps("data/20251129_results.zip", "data/20251129_results")
print(f"NT results available in '{to_dir}'.")

to_dir = "data/open-tyndp"
if not os.path.exists(to_dir):
    print(f"Unzipping data/open-tyndp.zip.")
    unzip_with_timestamps("data/open-tyndp.zip", "data/open-tyndp")
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# Interactive Exploration with PyPSA-Explorer

PyPSA-Explorer is an interactive web dashboard for visualizing and analyzing energy system networks. It provides:
- Energy balance analysis with time series and aggregated views
- Capacity planning visualizations by carrier and region
- Economic analysis showing CAPEX/OPEX breakdowns
- Interactive geographical network maps
- Multi-network visualisation support

Let's load the NT scenario results and explore them using PyPSA-Explorer.

In [ ]:
# Load NT scenario networks for comparison
base_path = "data/20251129_results/networks/"


def import_network(fn: str):
    n = pypsa.Network(fn)
    n.carriers.loc["none", "color"] = "#000000"
    return n


# Load networks directly into dictionary for PyPSA-Explorer
networks = {
    "NT 2030": import_network(base_path + "base_s_all___2030.nc"),
    "NT 2040": import_network(base_path + "base_s_all___2040.nc"),
}

PyPSA-Explorer can be launched in different ways depending on your environment:

- **Local Jupyter**: Use the terminal command (recommended) or inline display
- **Google Colab**: The dashboard launches inline, embedded directly in the notebook

Follow the instructions below for your environment.

In [ ]:
# Detect if running on Google Colab
try:
    from google.colab import output

    IN_COLAB = True
    print(f"This notebook is running on Google Colab !")
except ImportError:
    IN_COLAB = False
    print(f"This notebook is running locally !")

port = 8050

## For Local Users

If running locally, the **recommended approach** is to launch PyPSA-Explorer from the terminal:

```bash
pypsa-explorer data/20251129_results/networks/base_s_all___2030.nc:NT_2030 data/20251129_results/networks/base_s_all___2040.nc:NT_2040
```

This opens the dashboard in your default browser at http://localhost:8050.

**Alternative**: The cell below can launch the dashboard inline within the notebook, but the terminal method offers better performance.

In [ ]:
# Terminal method recommended
USE_TERMINAL = True  # Change to False if you want to launch inline display

if IN_COLAB or not USE_TERMINAL:
    # Local Jupyter: Inline display
    app = create_app(networks)
    app.run(jupyter_mode="tab", port=port, debug=False)

## For Google Colab Users

If running on Google Colab, start PyPSA-Explorer from the terminal before displaying it in the notebook.

Run this command in the Google Colab terminal:

```bash
pypsa-explorer data/20251129_results/networks/base_s_all___2030.nc:NT_2030 data/20251129_results/networks/base_s_all___2040.nc:NT_2040
```

In [ ]:
if IN_COLAB:
    # Google Colab: Use built-in Dash Jupyter support to display an iframe
    output.serve_kernel_port_as_iframe(port, height=1500)

**Tip for Colab users:** To view the dashboard in fullscreen mode, click the three dots (⋮) in the top-right corner of the output cell and select **"View output fullscreen"**.

## Using the Dashboard

Once the dashboard opens, explore these tabs:

1. **Energy Balance**: View production, consumption, and storage patterns
   - Switch between time series and aggregated views
   - Filter by energy carrier (electricity, hydrogen, etc.)
   - Filter by country/region

2. **Capacity**: Analyze installed capacities
   - Compare capacity buildout between 2030 and 2040 scenarios
   - View by technology type and region

3. **Economics**: Examine costs and revenues
   - CAPEX and OPEX breakdowns by technology
   - Regional cost comparisons
   - Investment requirements

4. **Network Map**: Visualize the geographical network
   - Interactive map with buses, lines, and generators
   - Zoom and pan to explore specific regions

**Tip:** Use the scenario buttons in the top-right corner to switch between NT 2030 and NT 2040.

# Benchmark Results

In Workshop 2, we introduced a benchmarking framework to systematically compare Open-TYNDP model outputs against TYNDP 2024. This framework helps us:

- **Identify discrepancies**: Compare demands, installed capacities, and generation volumes between model results and TYNDP targets
- **Quantify differences**: Calculate deviations by technology and investment year
- **Guide improvements**: Prioritize areas where the model requires refinement

Let's apply this framework to our NT 2030 and 2040 scenario results to understand where the current model aligns with or diverges from TYNDP expectations.

First, we'll define a helper function to display benchmarks.

In [ ]:
def show_benchmarks(
    fn: str,
    years: list = [2030, 2040],
    bench_path: str = "data/20251129_results/validation/graphics_s_all___all_years",
):
    try:
        images = [
            convert_from_path(Path(bench_path, f"{fn}_{y}.pdf"))[0] for y in years
        ]
    except PDFPageCountError:
        print("File not found, skipping...")
        return

    fig, axes = plt.subplots(1, 2)
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

## Overview

We begin with an overview of the Symmetric Mean Absolute Percentage Error (sMAPE) across all carriers. As a reminder, sMAPE measures the absolute magnitude of deviations while avoiding cancellation of negative and positive errors. This provides a high-level view of error magnitudes across all benchmarks, carriers, and planning horizons.

You can observe that some features have been significantly improved since we introduced the framework. However, there is still room for further improvement.

In [ ]:
display(
    convert_from_path(
        Path("data/20251129_results/validation/kpis_eu27_s_all___all_years.pdf")
    )[0]
)

## Final Energy Demand

Let's examine each benchmark category in detail, starting with final energy demand across all carriers. We observe that each carrier's demand is now broadly reproduced. However, the exact methodology details to fully match the TYNDP scope remain unclear, which explains the remaining discrepancies even though these demands are exogenous inputs.

In [ ]:
show_benchmarks("benchmark_final_energy_demand_eu27_cy2009")

## Electricity Demand

Electricity demand is an exogenous input to the model, taken directly from TYNDP 2024 data. Therefore, we achieve a perfect match between the model results and TYNDP 2024 for this input. Note that the final electricity demand equals the value of this exogenous input, which differs from the reference value.

In [ ]:
show_benchmarks("benchmark_elec_demand_eu27_cy2009")

## Power Capacities

Installed generation capacities are now converging toward TYNDP 2024 values. Both renewable and conventional capacities are fully integrated from the PEMMDB. Open-TYNDP closely reproduces TYNDP 2024, except for coal where large discrepancies between data sources persist.

CHP and small thermals, small-scale RES, and solar thermal technologies still need to be implemented. Currently, OCGT units are kept in the system and used as peaking units in Open-TYNDP because demand shedding is not yet implemented.

In [ ]:
show_benchmarks("benchmark_power_capacity_eu27_cy2009")

## Electricity Generation

Actual electricity generation differs slightly from the TYNDP 2024 generation mix. However, total generation values are closely aligned.

In [ ]:
show_benchmarks("benchmark_power_generation_eu27_cy2009")

We can also explore hourly timeseries balances.

In [ ]:
networks["NT 2030"].statistics.energy_balance.iplot.area(
    bus_carrier=["AC", "H2"],
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="bus_carrier",
    sharex=False,
    sharey=False,
    query="snapshot <= '2009-03-07' and snapshot >= '2009-03-01'",
)

## Hydrogen Demand

Hydrogen demand is primarily defined exogenously, so we expect good alignment between model results and TYNDP 2024. Note that hydrogen generation, in contrast, is determined endogenously by the optimization.

In [ ]:
show_benchmarks("benchmark_hydrogen_demand_eu27_cy2009")

## Hydrogen Supply

TODO

In [ ]:
show_benchmarks("benchmark_hydrogen_supply_eu27_cy2009")

## Methane Demand

Methane demand includes both exogenous consumption and endogenous demand for power generation and SMR (steam methane reforming). The exogenous consumption exactly reproduces the Supply Tool values. However, this is not yet aligned with the report values.

In [ ]:
show_benchmarks("benchmark_methane_demand_eu27_cy2009")

## Methane Supply

TODO

In [ ]:
show_benchmarks("benchmark_methane_supply_eu27_cy2009")

## Biomass Supply

TODO

In [ ]:
show_benchmarks("benchmark_biomass_supply_eu27_cy2009")

## Energy Imports

Energy imports include hydrogen, methane, liquids, and solids. Currently, the model cannot import biomass. Open-TYNDP values are overestimated because domestic production is not removed from the benchmarked values before comparison with TYNDP 2024.

In [ ]:
show_benchmarks("benchmark_energy_imports_eu27_cy2009")

# Modify assumptions

A key capability when working with any energy system model is adjusting input assumptions and observing how results respond. This is especially important for TYNDP models, where assumptions evolve over time and many configurations need testing and re-running.

In Open-TYNDP, there are several straightforward ways to modify assumptions and explore scenarios:
- **Input data**: Update relevant files in the `data` folder
- **Custom assumptions**: Override cost and technology assumptions via `custom_cost.csv`
- **Adjustments**: Make targeted changes to specific components through `scenario.config.yaml`
- **Custom constraints**: Add or adjust constraints directly in the `solve_network.py` script

## Modifying the input data

The first method to modify the model is by editing the input data directly.

Navigate to the `data` directory in the open-tyndp repository and locate the `tyndp_2024_bundle` folder.

In [ ]:
from scripts._helpers import display_tree

target_directory = "data/open-tyndp/data/tyndp_2024_bundle"
print("data")
display_tree(target_directory, max_depth=1)

You can browse and replace any input files with your own, provided they follow the **same format**.

## Custom assumptions

To add custom assumptions—such as capital costs, marginal costs, or technical parameters for specific technologies—use a long-format CSV table called `custom_cost.csv` in the `data` folder.

Open `custom_cost.csv` in the `data` directory. The file already contains example assumptions to illustrate the structure:

In [ ]:
custom_cost = pd.read_csv("data/open-tyndp/data/custom_costs.csv")
custom_cost

To add your own custom assumptions, insert a new row in this CSV. At minimum, specify the `planning_horizon`, `technology`, `parameter`, `value`, and `unit` for each entry. As shown in the file, you can also use the `all` argument to override a value for all technologies and/or all planning horizons.

These entries are automatically applied when you rerun the Open-TYNDP workflow.

## Adjustments

The third method to modify Open-TYNDP assumptions is making targeted changes to specific technologies in the system.

These are called `adjustments`, and you add them directly in the model configuration file for your scenarios.

You can find an example in the default configuration file:

In [ ]:
def display_code_lines(filename, language, start, end):
    with open(filename) as f:
        lines = f.readlines()
    return Code("".join(lines[start - 1 : end]), language=language)

In [ ]:
display_code_lines("data/open-tyndp/config/config.default.yaml", "yaml", 1069, 1076)

As shown, you can specify either a scaling factor or an absolute value for a given:
- Component type (e.g., Generator, Link)
- Carrier (i.e., technology)
- Attribute (e.g., marginal_cost, efficiency)

To make a manual adjustment to a scenario you're modeling, add an `adjustments` section to your `scenarios.tyndp.yaml`.

Let's open that file and examine its structure:

In [ ]:
display_code_lines("data/open-tyndp/config/scenarios.tyndp.yaml", "yaml", 1, 100)

## Custom constraints

You may also want to add custom constraints to the model. Since we haven't covered PyPSA constraints yet, we won't go into detail here, but we'll cover them comprehensively in a future workshop.

It's useful to know where you *would* add a custom constraint that isn't already represented through existing component parameters.

**Note**: An example of a standard PyPSA constraint is the `p_nom_max` attribute of a `Link` component. PyPSA automatically translates this into a binding upper-limit constraint on the capacity of an expandable `Link`.

To add a custom constraint, insert your own code directly into the model. This flexibility is one of the key advantages of working with an open-source setup.

Navigate to the `scripts` directory of the `open-tyndp` repository and open the `solve_network.py` script.

In [ ]:
display_code_lines("data/open-tyndp/scripts/solve_network.py", "Python", 1268, 1361)

In `solve_network.py`, you'll find a function called `extra_functionality`. This is where additional custom constraints are added to the model.

For instance, Open-TYNDP includes a custom constraint for TYNDP offshore hubs by calling the nested function `add_offshore_hubs_constraint`. As a reminder, this includes constraints to limit the expansion of DC and H2 wind farms at the same location, and to set the maximum potential per zone according to zone trajectories.

You can add your own constraints in the same location. Browse the constraints already implemented to understand the structure and style.

We'll cover how PyPSA constraints work in depth in a future workshop.

## Task 1: Add manual adjustments

Modify the existing `NT` scenario in `scenarios.tyndp.yaml` by adding the following manual adjustments:

- Increase the `marginal_cost` of all H2 imports by a factor of 1.5 (2030) and 1.3 (2040). The supply of imported H2 is included as a `Generator` component with the carrier name `import H2`.
- Change the `efficiency` of `H2 Electrolysis` to 78% for both 2030 and 2040. H2 Electrolysis is added as a `Link` component.
- Remove the initial capacity (`p_nom` and `p_nom_min`) of all `solar-pv-utility` generators for 2030.

Then rerun the model and explore the results in PyPSA-Explorer.

**Hint**: If you need a reminder on running the Snakemake workflow, refer to the notebook from our last workshop.

**Hint II**: Always start with a dry run first (add `-n` to your Snakemake command).

**Hint III**: As we only want to solve the networks, we can call the rule `solve_sector_networks` instead of the `all` rule.

# Solutions

## Task 1: Add manual adjustments

Add the following section to the `NT` scenario configuration in `scenarios.tyndp.yaml`:

```yaml
adjustments:
  sector:
    factor:
      Generator:
        import H2:
          marginal_cost:
            2030: 1.5
            2040: 1.3
        solar-pv-utility:
          p_nom:
            2030: 0.0
    absolute:
      Link:
        H2 Electrolysis:
          efficiency:
            2030: 0.78
            2040: 0.78
```

After adding the adjustments to `scenarios.tyndp.yaml`, we can use Snakemake to run the `open-tyndp` workflow with the standard `config.tyndp.yaml` configuration file.


For the workflow, we now need to install and activate the `open-tyndp` environment. The easiest way to do this, is by using the `pixi` environment manager which will take care of all of our dependencies in the background.
To install `pixi` locally for your operating system, you can follow the instructions on the `pixi` documentation page.

For execution on Google Colab we will install the Linux version into our runtime by executing:
```
!wget -qO- https://pixi.sh/install.sh | sh
```

We can then open a terminal window and as we learned previously, we need to navigate into the open-tyndp repository to launch the workflow:
```
cd data/open-tyndp
```

Once we are in our open-tyndp repository, we can activate our environment shell using pixi which will allow us to easily execute the workflow:
```
pixi shell -e open-tyndp
```

Now, we can start by performing a dry run:
```
snakemake -call solve_sector_networks --configfile config/config.tyndp.yaml -n
```

In [ ]:
# For execution on Google Colab we need to first install pixi before utilising the terminal to execute the workflow
# !wget -qO- https://pixi.sh/install.sh | sh

As shown, because this scenario has been run previously without the adjustments, Snakemake will only re-execute the rules affected by our manual adjustments.

Now execute the workflow by calling in your terminal:
```
snakemake -call solve_sector_networks --configfile config/config.tyndp.yaml
```

Now, we can launch the `PyPSA-Explorer` again and investigate the results of our new run. First we want to add our new results to our networks dictionary:

In [ ]:
# Load networks directly into dictionary for PyPSA-Explorer
networks.update(
    {
        "NT 2030 new": import_network(base_path + "base_s_all___2030.nc"),
        "NT 2040 new": import_network(base_path + "base_s_all___2040.nc"),
    }
)

In [ ]:
# Terminal method recommended
USE_TERMINAL = True  # Change to False if you want to launch inline display

if IN_COLAB or not USE_TERMINAL:
    # Local Jupyter: Inline display
    app = create_app(networks)
    app.run(jupyter_mode="tab", port=port, debug=False)

In [ ]:
if IN_COLAB:
    # Google Colab: Use built-in Dash Jupyter support to display an iframe
    output.serve_kernel_port_as_iframe(port, height=1500)